In [1]:
import tensorflow as tf

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, BatchNormalization, LSTM, Dense, concatenate, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, ResNet101

import numpy as np
import random
import matplotlib.pyplot as plt
import os
import cv2
import json

tf.__version__

'2.1.0'

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8196680103308608025
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4080628610892482692
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12663916536579097605
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14912199066
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16324043037359582911
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5"
]


In [3]:
def lossless_triplet_loss(y_true, y_pred, N=128, beta=128, epsilon=1e-8):
    """
    Implementation of the triplet loss function
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    N  --  The number of dimension 
    beta -- The scaling factor, N is recommended
    epsilon -- The Epsilon value to prevent ln(0)
    
    
    Returns:
    loss -- real number, value of the loss
    """
    anchor = tf.convert_to_tensor(y_pred[:,0:N])
    positive = tf.convert_to_tensor(y_pred[:,N:N*2]) 
    negative = tf.convert_to_tensor(y_pred[:,N*2:N*3])
    
    # distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),1)
    # distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),1)
    
    #Non Linear Values  
    
    # -ln(-x/N+1)
    pos_dist = -tf.math.log(-tf.divide((pos_dist),beta)+1+epsilon)
    neg_dist = -tf.math.log(-tf.divide((N-neg_dist),beta)+1+epsilon)
    
    # compute loss
    loss = neg_dist + pos_dist
    
    
    return loss

In [4]:
def res_base_network(in_dims, out_dims, activation='sigmoid'):
    model = Sequential()
    
    model.add(ResNet101(
    include_top=False, weights='imagenet', input_shape=in_dims, pooling='avg', classes=1000))

    model.add(Dense(1024, activation = 'relu'))
    model.add(Dense(out_dims, activation = activation))

    return model

In [5]:
def input_triplet_gen(gen, data_dir, num_classes, batch_size):
    
    while True:
        anchor_label, neg_label = random.sample(range(0, num_classes), 2)
        
        POS_DIR = os.path.join(data_dir, CLASS_DICT[anchor_label])
        NEG_DIR = os.path.join(data_dir, CLASS_DICT[neg_label])
        
        POS_filecount = len(os.listdir(os.path.join(POS_DIR, POS_DIR.split(os.path.sep)[-1])))    #POS_DIR 파일 개수 (뎁스 2번 들어가야해서 os.path.join~ 해주기)
        NEG_filecount = len(os.listdir(os.path.join(NEG_DIR, NEG_DIR.split(os.path.sep)[-1])))   #NEG_DIR 파일 개수
        
        if  POS_filecount < batch_size or NEG_filecount < batch_size:    #파일 개수가 batch_size 미만이면 처음으로 돌아가서 클래스 다시 선택
            continue

        anchor_gen = gen.flow_from_directory(POS_DIR, 
                                             target_size=(224, 224),
                                             batch_size=batch_size,
                                             color_mode='rgb')

        pos_gen = gen.flow_from_directory(POS_DIR,
                                          target_size=(224, 224),
                                          batch_size=batch_size,
                                          color_mode='rgb')

        neg_gen = gen.flow_from_directory(NEG_DIR, 
                                          target_size=(224, 224),
                                          batch_size=batch_size,
                                          color_mode='rgb')
    
#         print("Anchor: {} , Neg: {}".format(anchor_label, neg_label))
        X1i = anchor_gen.next()
        X2i = pos_gen.next()
        X3i = neg_gen.next()
        
        yield [X1i[0], X2i[0], X3i[0]], X1i[1]

In [6]:
data_dir = '../masked_moved2'
directory = os.listdir(data_dir)
CLASS_DICT = {k:v for k, v in enumerate(directory)}

gen = ImageDataGenerator(rescale=1./255)
num_classes = len(CLASS_DICT)
num_classes

5621

In [7]:
in_dims = (224, 224, 3)    # (28, 28, 1)
out_dims = 128

# Create the 3 inputs
anchor_in = Input(shape=in_dims, name='anchor')
pos_in = Input(shape=in_dims, name='positive')
neg_in = Input(shape=in_dims, name='negative')

# with tf.compat.v1.Session(config=config):
    # Share base network with the 3 inputs
base_network = res_base_network(in_dims, out_dims)
anchor_out = base_network(anchor_in)
pos_out = base_network(pos_in)
neg_out = base_network(neg_in)
    
merged_vector = concatenate([anchor_out, pos_out, neg_out], axis=-1)

# Define the trainable model
model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)

# for layer in base_network.layers[0].layers[:-21]:
#     layer.trainable=False

# for layer in base_network.layers[0].layers[-21:]:
#     layer.trainable=True

In [18]:
base_network.layers[0].trainable = False

In [21]:
#전체 freeze
base_network.layers[0].summary()

Model: "resnet101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

In [16]:
base_network.layers[0].trainable = True

In [20]:
for layer in base_network.layers[0].layers[:-21]:
    layer.trainable=False

for layer in base_network.layers[0].layers[-21:]:
    layer.trainable=True

In [13]:
cnt = 0

for layer in base_network.layers[0].layers:
    if layer.trainable == True:
        cnt += 1
        
print(cnt)

21


In [ ]:
base_network.layers[0].layers

In [ ]:
cnt = 0
for layer in base_network.layers[0].layers:
    if layer.trainable: cnt +=1
print('traninable layers: {}'.format(cnt))

In [11]:
model.compile(optimizer=Adam(learning_rate=0.0001, clipnorm=3.0),
                    loss=lossless_triplet_loss)

In [11]:
#일부 unfreeze
base_network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Model)            (None, 2048)              42658176  
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
Total params: 44,887,552
Trainable params: 2,229,376
Non-trainable params: 42,658,176
_________________________________________________________________


In [ ]:
for layer in base_network.layers[0].layers[:-21]:
    layer.trainable=True

for layer in base_network.layers[0].layers[-21:]:
    layer.trainable=True

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor (InputLayer)             [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
positive (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
negative (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 128)          44887552    anchor[0][0]                     
                                                                 positive[0][0]               

In [14]:
batch_size = 32
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='./trained/unfreeze_test/dummie.h5')]

# Training the model
H = model.fit_generator(input_triplet_gen(gen, data_dir, num_classes, batch_size),
              steps_per_epoch = 1, 
              epochs=5, 
              callbacks = my_callbacks)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 1 steps
Epoch 1/5
1/1 [==============================] - 15s 15s/step


ResourceExhaustedError: OOM when allocating tensor with shape[1,1,2048,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Fill] name: Adam/conv5_block2_1_conv/kernel/v/Initializer/zeros/

In [11]:
#weights 로드
model_weights_dir = './trained/unfreeze_test/dummie.h5'
model.load_weights(model_weights_dir)

In [13]:
batch_size = 32
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='./trained/py_test/{epoch:06d}-{loss:.4f}.h5')]

# Training the model
H = model.fit_generator(input_triplet_gen(gen, data_dir, num_classes, batch_size),
              steps_per_epoch = 1, 
              epochs=5, 
              callbacks = my_callbacks) 

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 1 steps
Epoch 1/5
1/1 [==============================] - 16s 16s/step - loss: 2.7260
Epoch 2/5
1/1 [==============================] - 1s 1s/step - loss: 2.6293
Epoch 3/5
1/1 [==============================] - 1s 1s/step - loss: 2.6185
Epoch 4/5
1/1 [==============================] - 1s 1s/step - loss: 2.6427
Epoch 5/5
1/1 [==============================] - 1s 1s/step - loss: 2.4556
